## 내장모듈

표준 모듈의 전체 집합은 너무나 방대해서 전부다 다루기에는 한계가 있다.  

그러나 몇몇 내장 패키지는 언어의 일부로 파이썬의 특징과 밀접한 관련이 있기 때문에 반드시 알아야한다.  

1. 함수 데코레이터를 정의하는 방법
2. with문의 사용법
3. time 모듈과 datetime모듈의 차이
4. 기본으로 제공되는 자료구조
5. 소수점이 중요할 땐 decimal

### 1. 함수 데코레이터를 정의하는 방법

파이썬은 데코레이터라는 특별한 문법을 가지고 있는데, 

데코레이터는 감싸고 있는 함수를 호출하기 전이나 후에추가로 코드를 실행하는 기능을 가지고 있다.  

이 기능은 시맨틱 강조, 디버깅, 함수 등록을 비롯해 여러상황에 유용하다.  

예를 들어 함수를 호출할 때 인수롸 반환 값을 출력하고 싶다고 하자. 특히, 재휘 호출에서 함수 호출 스택을 디버깅 할 때 도움이 된다.  

In [1]:
#case1. basic
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('{}({}, {}) -> {}'.format(func.__name__, args, kwargs, result))
        return result
    return wrapper

In [2]:
#case1. basic
@trace
def fibonacci(n):
    """n번재 피보나치 수열을 반환한다."""
    if n in (0, 1):
        return n
    return ( fibonacci(n-2) * fibonacci(n-1) )

In [3]:
fibonacci(3)
help(fibonacci)

fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 0
fibonacci((3,), {}) -> 0
Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



이 경우에는 __metadata가 복사되지 않아__ fibonacci에 대한 함수정보를 받아볼 수가 없다.  

이를 해결하기 위해서는 `functools의 wraps` 를 사용하면 된다.

In [4]:
#case2. Metadata copy version
from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('{}({}, {}) -> {}'.format(func.__name__, args, kwargs, result))
        return result
    return wrapper

In [5]:
#case2. Metadata copy version
@trace
def fibonacci(n):
    """n번재 피보나치 수열을 반환한다."""
    if n in (0, 1):
        return n
    return ( fibonacci(n-2) * fibonacci(n-1) )

In [6]:
fibonacci(3)
help(fibonacci)

fibonacci((1,), {}) -> 1
fibonacci((0,), {}) -> 0
fibonacci((1,), {}) -> 1
fibonacci((2,), {}) -> 0
fibonacci((3,), {}) -> 0
Help on function fibonacci in module __main__:

fibonacci(n)
    n번재 피보나치 수열을 반환한다.



요약하면 다음과 같다.  

- 데코레이터는 런타임에 한 함수로 다른 함수를 수정할 수 있게 해주는 파이썬 문법이다.  
- 데코레이터를 사용하면 객체 내부를 조사하는 help함수가 이상하게 작동하는 경우가 있는데, 이는 내장함수 `functools.wraps` 로 해결할 수 있다.

### 2. with문의 사용법

파이썬의 with문은 예를 들어 잠금을 사용하여 잠금이 설정되어 있는 동안만 들여 쓴 코드를 실행함을 나타낸다.  

In [7]:
from threading import Lock

lock = Lock()
with lock:
    print('Lock is held')

Lock is held


이 덕분에 아래에 해당하는 try/finally 구문을 사용하지 않고도 구현이 가능하다.  

```
lock.acquire()
try:
    print('Lock is held')
finally:
    lock.release()
```

이 때 with문을 제대로 사용하기 위해서 내장모듈 `contextlib`를 사용하면 객체와 함수를 with문에 쉽게 사용할 수 있도록 설계할 수 있다.  
`contextlib`는 `contextmanager`데코레이터를 포함한다.  

예를 들어 가끔씩 코드의 특정 영역에 더 많은 디버깅 로그를 넣고 싶다고 해보자. 여기서는 로깅 심각성 수준을 두 개로 로그를 남기는 함수를 만들어 본다.  

In [8]:
import logging

def my_function():
    logging.debug('Some Debug data')
    logging.error('Error log here')
    logging.debug('More Debug data')

In [9]:
my_function()

ERROR:root:Error log here


In [10]:
from contextlib import contextmanager

@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield
    finally:
        logger.setLevel(old_level)

이 때 yield 표현식은 with 블록의 내용이 실행되는 지점이다. with 블록에서 일어나는 모든 예외를 yield 표현식이 다시 일으킴으로 헬퍼함수로 처리할 수 있다.

In [11]:
print('Before: ')
my_function()

with debug_logging(logging.DEBUG):
    print('Inside: ')
    my_function()

ERROR:root:Error log here
DEBUG:root:Some Debug data
ERROR:root:Error log here
DEBUG:root:More Debug data


Before: 
Inside: 


In [13]:
with open('./dataset/my_output43.txt', 'w') as handle:
    handle.write('This is some data!')

In [16]:
@contextmanager
def log_level(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield logger
    finally:
        logger.setLevel(old_level)    

In [17]:
with log_level(logging.DEBUG, 'my-log') as logger:
    logger.debug('This is my message')
    logging.debug('This will not pring')
    

DEBUG:my-log:This is my message


In [18]:
logger = logging.getLogger('my-log')
logger.debug('Debug will not print')
logger.error('Error will print')

ERROR:my-log:Error will print


이렇게 사용하는 경우 try/finally 구문을 재사용할 수 있게 된다.

요약하면 다음과 같다.
- with문을 이용하면 try/finally 문을 재사용할 수 있고, 코드를 깔끔하게 만들 수 있다.  
- contextlib의 contextmanager를 사용하면 직접 작성한 함수를 with문에 쉽게 사용할 수 있다.
- contextmanager에서 넘겨준 값은 as 부분이고, 값을 반환하려면 코트에서 특별한 컨텍스트에 직접접근하려고 할 때 유용하다.  

### 3. time 모듈과 datetime모듈의 차이

일반적으로 하는 기능은 같지만, time 보다는 __datetime__ 모듈을 사용하는 것인 안전하다. 그 이유는 time의 경우 운영체제와의 호환성에 의해서 불안정한 경우가 다수 발생하기 때문이다.  

그렇기 때문에 time 모듈을 사용하더라도 UTC 시간을 지역시간으로 변경하는 정도의 용도로만 사용하도록 하자.  

In [20]:
from time import localtime, strftime

now = 1407694710
local_tuple = localtime(now)
time_format = '%Y-%m-%d %H:%M:%S'
time_str = strftime(time_format, local_tuple)
print(time_str)

2014-08-11 03:18:30


In [21]:
from time import mktime, strptime

time_tuple = strptime(time_str, time_format)
utc_now = mktime(time_tuple)
print(utc_now)

1407694710.0


#### UTC to local_time
다음은 파이썬의 시간을 UTC로 불러와서, 나의 지역시간으로 변경하는 코드이다.  
datetime 모듈을 time 모듈과 다르게 한 지역 시간을 다른 지역 시간으로 신뢰성 잇게 변경한다. 하지만 tzinfo 클래스와 관련 매서드를 이용한 시간대 변환기능만 제공한다. 

빠진 부분은 UTC 이외의 시간대 정의다. 다행이도 이 부분은 `pytz` 모듈로 해결하고 있다.  pytz는 필요한 모든 시간대에 대한 정의를 담은 전체 데이터베이스를 포함한다.  
pytz를 효과적으로 사용하기 위해서는 항상 지역시간을 UTC로 변경후에 사용해야한다.  

In [22]:
from datetime import datetime, timezone

now = datetime(2014, 8, 10, 18, 18, 30)
now_utc = now.replace(tzinfo=timezone.utc)
now_local = now_utc.astimezone()
print(now_local)

2014-08-11 03:18:30+09:00


이를 다시 쉽게 UTC시간으로 바꿀수도 있다.

In [26]:
from datetime import datetime

time_str = '2014-08-10 11:18:30'
now = datetime.strptime(time_str, time_format)
time_tuple = now.timetuple()
utc_now = mktime(time_tuple)
print(utc_now)

1407637110.0


In [31]:
#local_time to UTC
import pytz

arrival_nyc = '2014-05-01 23:33:24'
nyc_dt_naive = datetime.strptime(arrival_nyc, time_format)
eastern = pytz.timezone('US/Eastern')

nyc_dt = eastern.localize(nyc_dt_naive)
utc_dt = pytz.utc.normalize(nyc_dt.astimezone(pytz.utc))
print(utc_dt)

2014-05-02 03:33:24+00:00


In [33]:
#UTC to another local time

pacific = pytz.timezone('US/Pacific')
sf_dt = pacific.normalize(utc_dt.astimezone(pacific))
print(sf_dt)

nepal = pytz.timezone('Asia/Katmandu')
nepal_dt = nepal.normalize(utc_dt.astimezone(nepal))
print(nepal_dt)

2014-05-01 20:33:24-07:00
2014-05-02 09:18:24+05:45


### 4. 기본으로 제공되는 자료구조


#### Deque(Double-ended que)

In [36]:
from collections import deque

fifo  = deque()
fifo.append(5)
fifo.append(1)
x = fifo.popleft()
print(x)
print(fifo)

5
deque([1])


#### OrderedDict

딕셔너리를 마치 순서가 있는 이터레이터처럼 사용하고자 할 때 유용하다.  

In [37]:
from collections import OrderedDict

a = OrderedDict()
a['foo'] = 1
a['bar'] = 2

b = OrderedDict()
b['foo'] = 'red'
b['bar'] = 'blue'

for value1, value2 in zip(a.values(), b.values()):
    print(value1, value2)

1 red
2 blue


#### 힙큐(Heapque)

힙은 우선순위 큐 를 유지하는 유용한 자료이다. 또한 연산에 걸리는 시간이 리스트 길이에 비례하려 로그형태로 증가하기 때문에, 연산적 장점을 가지고 있다. 

In [40]:
from heapq import *

a = []
heappush(a, 5)
heappush(a, 3)
heappush(a, 7)
heappush(a, 4)

print(heappop(a), heappop(a), heappop(a), heappop(a))

3 4 5 7


In [41]:
a = []
heappush(a, 5)
heappush(a, 3)
heappush(a, 7)
heappush(a, 4)

print('Before: ', a)
a.sort()
print('After: ', a)

Before:  [3, 4, 7, 5]
After:  [3, 4, 5, 7]


#### 바이섹션(Bisection)
바이섹션 또한 연산에 걸리는 시간이 리스트 길이에 비례하려 로그형태로 증가하기 때문에, 연산적 장점을 가지고 있다. 

특히 인덱스를 반환한다고 생각했을 때, 100만의 경우 list.index는 100만에 비례하는 반면, 바이섹션의 경우log(100만)~14 정도의 탐색구간으로 엄청난 탐색 속도를 보인다.  

In [50]:
x = list(range(10**6))

#list index method
start = datetime.now()
i = x.index(991234)
end = datetime.now()
print('list indexing method: ',(end-start).total_seconds())


#Bisection index method
from bisect import bisect_left
start = datetime.now()
i = bisect_left(x, 991234)
end = datetime.now()
print('Bisection method: ',(end-start).total_seconds())

list indexing method:  0.016004
Bisection method:  0.0


### 5. 소수점이 중요할 땐 decimal